In [1]:
from Scripts.import_data import *

import_companies_balance_sheet()

data_dict = {}
data_dict = import_companies_balance_sheet()

#data_dict = load_csv('./Data/companies_balance_sheet.csv')

In [2]:
print(data_dict.keys())

dict_keys(['11BIT', '3LPSA', '3RGAMES', 'AB', 'AC', 'ACTION', 'ADIUVO', 'AGORA', 'AGROPUBL', 'AGROWILL', 'AIGAMES', 'AILLERON', 'AIRWAY', 'ALLEGRO', 'ALUMETAL', 'AMBRA', 'AMICA', 'AMPLI', 'AMREST', 'ANSWEAR', 'APATOR', 'APLISENS', 'APSENERG', 'ARCTIC', 'ARTERIA', 'ARTIFEX', 'ASBIS', 'ASMGROUP', 'ASSECOBS', 'ASSECOP', 'ASSECOSE', 'ASTARTA', 'ATENDE', 'ATLANTA', 'ATMG', 'ATREM', 'AUTOPART', 'AZOTY', 'BAHOLDING', 'BEDZIN', 'BENEFIT', 'BERLING', 'BETACOM', 'BIGCHEESE', 'BIOCELTIX', 'BIOMAXIM', 'BIOMED', 'BIOPLAN', 'BIOTON', 'BMW', 'BOGDANKA', 'BOOMBIT', 'BORYSZEW', 'BOWIM', 'BRAND24', 'BRASTER', 'BUDIMEX', 'BUMECH', 'CAPTORTX', 'CCC', 'CDPROJEK', 'CDRL', 'CENNOWTE', 'CEZ', 'CFIHOL', 'CIECH', 'CIGAMES', 'CITYSERV', 'CLNPHARMA', 'COALENER', 'COGNOR', 'COMARCH', 'COMPERIA', 'COMPREMUM', 'COMP_W', 'CORMAY', 'CREEPYJAR', 'CREOTECH', 'CYFPOLS', 'DADELO', 'DATAWALK', 'DEBICA', 'DECORA', 'DEKPOL', 'DELKO', 'DIGITANET', 'DIGITREE', 'DINOPL', 'DOMDEVEL', 'DREWEX', 'DROZAPOL', 'EDINV', 'EFEKT', 'EKO_